In [1]:
from datetime import date, datetime

import pandas as pd
import numpy as np
import pyomo.environ as pyo
from thefuzz import process, fuzz



from app.model import UnitCommitmentModel, DispatchConfig

In [2]:
config = DispatchConfig(
    dispatch_type="preideal"
)

In [3]:
DISPATCH_DATE = date(2024,6,9)
# DISPATCH_DATE = date(2024,5,25)
# DISPATCH_DATE = date(2024,8,10)
# DISPATCH_DATE = date(2024,2,15)

# 1. Load data

In [4]:
# dispo = pd.read_csv('data/DispoCome_resource.csv', parse_dates=["datetime"], engine="pyarrow")
dispo = pd.read_csv('data/dispo_declarada.csv', parse_dates=["datetime"], engine="pyarrow")
ofertas = pd.read_csv('data/ofertas.csv', parse_dates=["Date"], engine="pyarrow")
demanda = pd.read_csv('data/demaCome.csv', parse_dates=["datetime"], engine="pyarrow")
agc_asignado = pd.read_csv('data/agc_asignado.csv', parse_dates=["datetime"], engine="pyarrow")


In [5]:
import re

price_pattern = r"P(\d+)"
dispo_pattern = r"DISCONF(\d+)"
# matches = re.findall(pattern, "DISCONF7 DISCONF8 DISCONF1")
# matches = re.findall(pattern, "P14 P P7")
# print(matches)  # Output: ['7', '8', '1']

In [ ]:
output = []
MO = []
CC = {}
cc_price = {}
cc_dispo = {}
prices = {}
with open(f"data/OFEI{DISPATCH_DATE.month:0>2}{DISPATCH_DATE.day:0>2}.txt", "r") as file:
    for line in file.readlines():
        line = line.strip()
        if "PAP" in line:
            output.append(line)
        if "MO" in line:
            mo_line = line.split(",")
            if len(mo_line) > 2 and "MO" in mo_line[1]:
                MO.append(mo_line)
        if (conf:=re.findall(price_pattern, line)) and "CC" in line:
            fline = line.split(",")
            cc_price[f"{fline[0].strip()}_{conf[0]}"] = float(fline[2])
            if CC.get(fline[0].strip()):
                CC[fline[0].strip()].append(f"{fline[0].strip()}_{conf[0]}")
            else:
                CC[fline[0].strip()] = [f"{fline[0].strip()}_{conf[0]}"]
        # Disponibilidad CC
        if (conf:=re.findall(dispo_pattern, line)) and "CC" in line:
            fline = line.split(",")
            cc_dispo[f"{fline[0].strip()}_{conf[0]}"] = [int(disp) for disp in fline[2:]]

        # Extract prices
        if "P" in line:
            pri = line.split(",")
            if len(pri) == 3 and " P" in pri[1] and not "u" in pri[1].lower() and not "a" in pri[1].lower():
                prices[pri[0]] = float(pri[2])*1E-3

            
            
precio_arranque = pd.DataFrame(
    [
        line.split(",")
        for line in output
        if not "usd" in line.lower()
    ],
    columns=["resource", "type", "price"]
)
precio_arranque["price"] = precio_arranque["price"].astype(float)

# Minimo operativo
minimo_operativo = pd.DataFrame(
    MO,
    columns=["resource", "type", ] + list(range(24))
)
minimo_operativo = minimo_operativo.set_index(["resource", "type"]).stack().reset_index()
minimo_operativo.columns = ["resource", "type", "hour", "minimo_operativo"]
minimo_operativo["datetime"] = pd.to_datetime(DISPATCH_DATE) + pd.to_timedelta(minimo_operativo["hour"], unit="h")
minimo_operativo["minimo_operativo"] = minimo_operativo["minimo_operativo"].astype(float)
minimo_operativo

In [7]:
dispo = dispo[
    (dispo.datetime.dt.date == DISPATCH_DATE) &
    (dispo["resource_name"].notnull())
]
dispo = dispo.drop_duplicates(subset=["resource_name", "datetime"])
ofertas = ofertas[ofertas.Date.dt.date == DISPATCH_DATE]
agc_asignado = agc_asignado[agc_asignado["datetime"].dt.date == DISPATCH_DATE]
demanda = demanda[demanda["datetime"].dt.date == DISPATCH_DATE]

In [8]:
price_bid_map = {
    gen: process.extractOne(
        query=gen.lower(),
        choices=dispo["resource_name"].unique(),
        scorer=fuzz.token_sort_ratio,
        processor=lambda x: x.lower().replace(" ", ""),
        score_cutoff=70,
    )[0]
    for gen in prices.keys()
}
prices = {
    price_bid_map[gen]: price
    for gen, price in prices.items()
}

In [ ]:
ofertas["Value"] = ofertas.apply(lambda x: prices.get(x["resource_name"],float(x["Value"])), axis=1)
ofertas

In [10]:
# ofertas.loc[ofertas["resource_name"].str.contains("TEBSA"),"Value"] = 500.000
# ofertas[ofertas["resource_name"].str.contains("TEBSA")]

In [11]:
# import numpy as np
# dispo.loc[dispo["resource_name"].str.contains("VALLE"),"dispo"] = np.array([239,  1,  1,  1,  1,  1,  1,  1,  1,  1,  239,  239,  239,  239,  239,  239,  239,  239,  239,  239,  239,  239,  239,  239])*1E3
# ofertas.loc[ofertas["resource_name"].str.contains("VALLE"),"Value"] = 500.000

In [12]:
# ofertas.loc[ofertas["resource_name"].str.contains("TEBSA"),"Value"] = 1514.537

In [13]:
# ofertas.head()

# 3. USING CC

In [14]:
# DROP previous CC
CC_MAP = {
    gen: process.extractOne(
        query=gen.lower(),
        choices=dispo["resource_name"].unique(),
        scorer=fuzz.partial_token_sort_ratio,
        processor=lambda x: x.lower().replace(" ", ""),
        score_cutoff=70,
    )[0]
    for gen in CC.keys()
}
CC_MAP

dispo = dispo[~dispo["resource_name"].isin(list(CC_MAP.values()))]
ofertas = ofertas[~ofertas["resource_name"].isin(list(CC_MAP.values()))]

In [15]:
# INCLUDING CC RESOURCE in DISPO and OFERTAS
new_cc_resources = pd.DataFrame(cc_dispo).stack().reset_index()
new_cc_resources.columns = ["hours", "resource_name", "dispo"]
new_cc_resources["dispo"] = new_cc_resources["dispo"]*1e3
new_cc_resources["hours"] = new_cc_resources["hours"].astype(int)
new_cc_resources["datetime"] = pd.to_datetime(DISPATCH_DATE) + pd.to_timedelta(new_cc_resources["hours"], unit="h")
new_cc_resources["gen_type"] = "TERMICA"
new_cc_resources["dispatched"] = "DESPACHADO CENTRALMENTE"
new_cc_resources["company_activity"] = "GENERACIÓN"
new_cc_resources.pop("hours")


# OFERTAS

new_cc_bid = pd.DataFrame(cc_price, index=[1]).stack().reset_index(drop=False)
new_cc_bid.columns = ["index_", "resource_name", "Value"]
new_cc_bid["Value"] = new_cc_bid["Value"].apply(lambda x: x*1E-3)
# new_cc_bid["datetime"] = pd.to_datetime(DISPATCH_DATE) + pd.to_timedelta(new_cc_bid["hours"], unit="h")
new_cc_bid["resource_gen_type"] = "TERMICA"
new_cc_bid["Date"] = DISPATCH_DATE
# new_cc_bid["dispatched"] = "DESPACHADO CENTRALMENTE"
# new_cc_bid["company_activity"] = "GENERACIÓN"
_ = new_cc_bid.pop("index_")



In [ ]:
dispo = pd.concat([dispo, new_cc_resources], axis=0)
ofertas = pd.concat([ofertas, new_cc_bid], axis=0)
ofertas

In [17]:
major_generators = ofertas.resource_name.unique()
generators = dispo.resource_name.unique()
timestamps = demanda["datetime"].to_dict().values()
# fuel_generators = dispo.query('resource_name in @major_generators and gen_type=="TERMICA"').resource_name.unique()
fuel_generators = dispo[
    (dispo["resource_name"].isin(major_generators)) &
    (dispo["gen_type"] == "TERMICA")
].resource_name.unique()

## 1.2. Get startup/shutdown costs

In [ ]:
MO_map = {
    gen: results[0]
    for gen in minimo_operativo.resource.unique()
    if (results := process.extractOne(
        query=gen.lower(),
        choices=generators,
        # choices=major_generators.tolist(),
        scorer=fuzz.token_sort_ratio,
        processor=lambda x: x.lower().replace(" ", ""),
        score_cutoff=70,
    ))
}
minimo_operativo["resource"] = minimo_operativo["resource"].apply(lambda x: MO_map.get(x, x))
minimo_operativo

In [19]:
generators_pap_map = {
    gen: process.extractOne(
        query=gen.lower(),
        choices=precio_arranque.resource.unique(),
        scorer=fuzz.partial_token_sort_ratio,
        processor=lambda x: x.lower().replace(" ", ""),
        score_cutoff=70,
    )[0]
    for gen in fuel_generators
}

cold_start = {}
for gen in fuel_generators:
    gen_name_mapped = generators_pap_map[gen]
    gen_pap=precio_arranque[
        (precio_arranque["resource"] == gen_name_mapped) &
        (precio_arranque.type.str.contains("F"))
    ]["price"].values[0]
    cold_start[gen] = float(gen_pap)
  


# 1.3. Condiciones Iniciales

In [20]:
# with open(f"data/dCondIniP{DISPATCH_DATE.month:0>2}{DISPATCH_DATE.day:0>2}.txt", "r") as file:
#     data = file.readlines()
#     data = [line.strip().split(",") for line in data]
#     headers = data.pop(0)
# data = pd.DataFrame(data, columns=headers)
# data

In [21]:
# Valores en MWh
Pmax = dispo.query("resource_name in @generators").set_index(["resource_name","datetime"]).sort_index()["dispo"]*1E-3
Pmin = minimo_operativo.set_index(["resource", "datetime"]).sort_index()["minimo_operativo"]
beta = ofertas.query("resource_name in @generators").set_index(["resource_name"]).sort_index()["Value"]*1E3
agc_indexed = agc_asignado.set_index(["recurso", "datetime"])["agc"]*1E-3

# Pmax.loc[agc_indexed.index] = Pmax.loc[agc_indexed.index] -  agc_indexed

In [22]:
demand_pronos =pd.read_csv(f"data/preideal_dispatch_{DISPATCH_DATE}.txt", header=None)
demand_pronos = demand_pronos.iloc[:, 1:].sum().values

In [23]:
demand_pronos = dict(zip(demanda["datetime"], demand_pronos))

In [24]:
set_data = {
    "G": fuel_generators,
    "T": timestamps,
    "I": generators,
    # "combined_cycle": list(CC.keys()),
    # "excluded_resource": CC,
}

param_data = {
    # "Pmax" : Pmax,
    "Pmax" : Pmax.apply(lambda x: np.round(x,0)),
    "Pmin" : Pmin,
    "beta" : beta,
    "cold_start" : cold_start,
    "demand" : demand_pronos,
    # "demand" : demanda.set_index("datetime")["dema"],
}


In [25]:
model = UnitCommitmentModel(config=config)
model.create_model(set_data=set_data, param_data=param_data)


results = model.solve(solver="cbc")
# results = model.solve(solver="cplex", executable="solver/cplex")

# model._model.z.fix()

# results = model.solve(solver="cplex", executable="solver/cplex")

In [ ]:
expr = model._model.objective.expr()
print(f"F.obj: {expr:,.2f}")

In [27]:
mpo_xm = pd.read_csv(f"data/preideal_price_{DISPATCH_DATE}.txt", header=None)
mpo_xm = mpo_xm.iloc[0,1:].values

In [ ]:
mpo_xm

In [ ]:
MPO = {
    ke.index(): pyo.value(dual_)
    for ke, dual_ in model._model.dual.items()
    if "power_balance" in ke.name
}
MPO

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 1)
pd.DataFrame(data=MPO, index=["MPO"]).T.plot(kind="line",ax=ax)
pd.DataFrame(data=mpo_xm, index=timestamps, columns=["MPO_XM"]).plot(kind="line", ax=ax, linestyle='--')

In [ ]:
model._model.pout.display()

In [ ]:
model._model.beta.display()

In [ ]:
model._model.Pmax.display()

In [ ]:
model._model.pout.display()

In [ ]:
model._model.z.display()

In [ ]:
model._model.zup.display()

In [ ]:
model._model.pout.display()

In [ ]:
for k,v in model._model.Pmin["PAIPA 2",:].get_item:
    print(k, v)